# **Installing the dependencies**

In [ ]:
#https://github.com/graykode/nlp-tutorial/blob/master/5-1.Transformer/Transformer.py#L45

In [ ]:
!pip install transformers torch
!pip install -U SentencePiece
!pip install accelerate -U
!pip install datasets

# **Connecting google colab to drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Importing the dependencies**

In [ ]:
import json
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification, T5Tokenizer, T5ForSequenceClassification,TrainingArguments, Trainer
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import random
from datasets import Dataset
import numpy as np
import math
import pandas as pd
from collections import Counter
import sys
import json
# visualization libraries
import matplotlib.pyplot as plt
import numpy as np

# pytorch libraries
import torch # the main pytorch library
import torch.nn as nn # the sub-library containing Softmax, Module and other useful functions
import torch.optim as optim # the sub-library containing the common optimizers (SGD, Adam, etc.)

# huggingface's transformers library
#from transformers import RobertaForTokenClassification, RobertaTokenizer
from transformers import RobertaForTokenClassification, RobertaTokenizer
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer

# huggingface's datasets library
from datasets import load_dataset

# the tqdm library used to show the iteration progress
import tqdm
tqdmn = tqdm.notebook.tqdm#tqdm.tqdm# tqdm.notebook.tqdm

from datasets import load_dataset
import os
from transformers import get_scheduler
from datasets import Dataset
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import copy


## Data format

2 json files - `surprise.data` contains utterances, `surprise.solution` contains corresponding intents

Format of `surprise.data`

```json
{"indoml_id": "surprise|11109", "id": "11109", "utt": "Can I make a reservation at Buffalo Wild Wings?"}
{"indoml_id": "surprise|11051", "id": "11051", "utt": "Can I book a table for tonight at Bella Vita?"}
....
```

Format of `surprise.solution` (contains 150 labels)

```json
{"indoml_id": "surprise|11109", "intent": "accept reservations"}
{"indoml_id": "surprise|11051", "intent": "accept reservations"}
....
```


# **Loading the data and solution of the surprise dataset**

In [ ]:
# Load the data from the JSON files
with open('/home/naive123/nlp/Sumit/massive/suprise/surprise_data/surprise.data', 'r') as data_file:
    data = [json.loads(line) for line in data_file] # converting string to object using json.loads

with open('/home/naive123/nlp/Sumit/massive/suprise/surprise_data/surprise.solution', 'r') as solution_file:
    solutions = [json.loads(line) for line in solution_file] # converting string to object using json.loads

# with open('/content/drive/MyDrive/Intent_classification_IIT_BOMBAY/Dataset/massive_test_phase2_data/massive_test.data', 'r') as test_file:
#     test = [json.loads(line) for line in test_file]



In [ ]:
!nvidia-smi


Wed Oct 11 13:07:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:81:00.0 Off |                  Off |
| 34%   31C    P8     6W / 260W |    161MiB / 24217MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **loading and perform preprocessing on blind dataset(Testing dataset)**

In [ ]:
solution_path='/home/naive123/nlp/Sumit/massive/suprise/massive_test.data'
with open(solution_path,'r') as file:
	files=file.readlines()

test_list=[]
for line in files:
	line=line.strip('\n')
	line=json.loads(line)
	test_list.append(line)


In [ ]:
test_list[:20]

[{'indoml_id': 1,
  'utt': 'Kindly show me my entire transaction record from the beginning of this year.'},
 {'indoml_id': 2, 'utt': 'Could you help me get this thing delivered today?'},
 {'indoml_id': 3, 'utt': 'Can you join forces with my mobile?'},
 {'indoml_id': 4,
  'utt': 'Can I get an update on the interest rate for my TD Bank savings account?'},
 {'indoml_id': 5, 'utt': "Absolutely! I'd love some more coffee too."},
 {'indoml_id': 6,
  'utt': 'What are the necessary tools required to jump-start my vehicle?'},
 {'indoml_id': 7,
  'utt': 'how long does it take for my credit score to recover from a bankruptcy filing?'},
 {'indoml_id': 8, 'utt': 'Could you please play the next track?'},
 {'indoml_id': 9,
  'utt': 'How do I go about increasing my credit limit on my Discover it cashback card?'},
 {'indoml_id': 10,
  'utt': 'Kindly provide me with the exact date of expiration of my Capital One card.'},
 {'indoml_id': 11,
  'utt': 'may I ask if you would prefer to communicate in swahil

In [ ]:
print(data[0]) # data of the surpirse dataset having id and the utterances

{'indoml_id': 'surprise|11109', 'id': '11109', 'utt': 'Can I make a reservation at Buffalo Wild Wings?'}


In [ ]:

print(solutions[0]) # Solutions of the surpise dataset having id and intent for the previous teerances

{'indoml_id': 'surprise|11109', 'intent': 'accept reservations'}


In [ ]:
# Create a dictionary to map indoml_id to intents
intent_map = {item['indoml_id']: item['intent'] for item in solutions}

# Split data into train and test sets (2:1) stratified by intent
indoml_ids = [item['indoml_id'] for item in data]
intents = [intent_map[indoml_id] for indoml_id in indoml_ids]
"""utterances(features) for the dataset"""
utt = [item['utt'] for item in data]

num_classes = len(set(intents)) # total number of unique intents are the total number of classes
print("Number of classes")
print(num_classes)

"""Splitting the dataset into train and test set"""
train_data, test_data, train_labels, test_labels = train_test_split(
    utt, intents, test_size=0.10, random_state=42, stratify=intents
)

Number of classes
150


In [ ]:
train_labels[0:5], len(train_labels) # these are basically our intents corrosponding to the utterances for trainin g data(5%) of the data

(['interest rate',
  'restaurant suggestion',
  'change language',
  'shopping list',
  'oil change how'],
 2023)

In [ ]:
train_labels[0:5]

['interest rate',
 'restaurant suggestion',
 'change language',
 'shopping list',
 'oil change how']

In [ ]:
train_data[:5]

['please tell me what i can expect my us bank interest rate to be.',
 'Can you recommend a great Italian restaurant nearby?',
 'Can you communicate with me in Japanese?',
 'Is there a way to organize my shopping list by categories?',
 'Can you walk me through the process of changing my oil?']

In [ ]:
test_labels[0:5], len(test_labels) # these are basically our intents corrosponding to the utterances for testing data(5%) of the data

(['change language', 'yes', 'how old are you', 'who made you', 'smart home'],
 225)

In [ ]:
intents
intents_set=set(intents) # basically represent the total no. of disntics intents(labels) in our surprise dataset(150)
print(intents_set)
len(intents_set)

{'restaurant reviews', 'definition', 'next holiday', 'whisper mode', 'damaged card', 'cook time', 'are you a bot', 'what is your name', 'w2', 'do you have pets', 'nutrition info', 'pay bill', 'travel notification', 'redeem rewards', 'flip coin', 'meeting schedule', 'restaurant suggestion', 'tire pressure', 'timer', 'alarm', 'change user name', 'maybe', 'pto used', 'date', 'fun fact', 'what can i ask you', 'order', 'insurance', 'carry on', 'todo list', 'uber', 'recipe', 'find phone', 'improve credit score', 'change ai name', 'freeze account', 'book hotel', 'change volume', 'restaurant reservation', 'timezone', 'next song', 'todo list update', 'calendar', 'taxes', 'travel suggestion', 'rewards balance', 'gas', 'cancel reservation', 'income', 'report fraud', 'international fees', 'reset settings', 'min payment', 'confirm reservation', 'travel alert', 'food last', 'transactions', 'what are your hobbies', 'rollover 401k', 'change speed', 'cancel', 'schedule maintenance', 'plug type', 'credi

150

In [ ]:
id2label={} # id2label mapping
label2id={} # label2id mapping
for i, intent in enumerate(intents_set):
  id2label[i] = intent
  label2id[intent]=i



In [ ]:
"""Convert labels to numeric values from label2id mapping"""
train_int_labels = [label2id[label] for label in train_labels]
test_int_labels = [label2id[label] for label in test_labels]
test_int_labels[0:5]

[124, 74, 119, 106, 129]

In [ ]:
train_dict={'utt':train_data, "intent":train_int_labels}
test_dict={'utt':test_data, "intent":test_int_labels}
test_dict

{'utt': ['Kindly alter my language preference to Japanese.',
  "Absolutely, I couldn't agree more!",
  'how old do you think you will be next year, ai?',
  'who is credited with creating your robotics software',
  "What's the wifi password for my living room lamp?",
  'Can you please tell me how much I need to pay for my water bill this month?',
  'I would like to add "Happy" by Pharrell Williams to my feel-good playlist.',
  'I need an alarm to go off at 10 pm to wind down.',
  'How do you say "goodnight" in Italian?',
  "I'd love to know more about your pets.",
  'Can you tell me if my tires are properly inflated?',
  'How good is the APR on my credit card?',
  'I am interested in renting an electric car in Seattle for a road trip along the Pacific Coast Highway. Can you provide more information on these types of vehicles?',
  'What is the result of subtracting 19 from 37?',
  'i want to talk to ben.',
  'How do I go about checking my account balance?',
  'Have I been spending too mu

#**label encoding- converting labels to numeric values**

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
intents_int=label_encoder.fit_transform(intents)
intents_int, intents_int.shape

(array([  0,   0,   0, ..., 149, 149, 149]), (2248,))

In [ ]:
print(list(id2label.items())[:5])
print('\n')
print(list(label2id.items())[:5])


[(0, 'restaurant reviews'), (1, 'definition'), (2, 'next holiday'), (3, 'whisper mode'), (4, 'damaged card')]


[('restaurant reviews', 0), ('definition', 1), ('next holiday', 2), ('whisper mode', 3), ('damaged card', 4)]


In [ ]:
seed=50
torch.manual_seed(seed)
num_labels = 150
# model_max_length = 64
lr=6e-4
n_epochs = 30
batch_size= 32
min_batch_size=2
batch_size_c = batch_size / min_batch_size
early_stoping_patience = 6 # if valid loss will increase 3 consecutive times with a margin then training will stop.
# initial_model = "xlm-roberta-large"

# **Extra Encoder layer**

In [ ]:
n_layers=1
d_model = 1024
n_heads=1
d_k=512
d_v=512
d_ff=512
batch_size_1=min_batch_size
hidden_dropout_prob=0.1
num_labels=150

class GELU(nn.Module):

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

class Embedding(nn.Module):
   def __init__(self):
       super(Embedding, self).__init__()
       self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
       self.pos_embed = nn.Embedding(maxlen, d_model)  # position embedding
       self.seg_embed = nn.Embedding(n_segments, d_model)  # segment(token type) embedding
       self.norm = nn.LayerNorm(d_model)

   def forward(self, x, seg):
       seq_len = x.size(1)
       pos = torch.arange(seq_len, dtype=torch.long)
       pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
       embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
       return self.norm(embedding)

class MultiHeadAttention(nn.Module):
   def __init__(self):
       super(MultiHeadAttention, self).__init__()
       self.W_Q = nn.Linear(d_model, d_k * n_heads)
       self.W_K = nn.Linear(d_model, d_k * n_heads)
       self.W_V = nn.Linear(d_model, d_v * n_heads)
       self.linear_layer = nn.Linear(n_heads * d_v, d_model)
       self.layer_norm = nn.LayerNorm(d_model)

   def forward(self, Q, K, V, attn_mask):
       # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
       residual, batch_size = Q, Q.size(0)
       # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
       q_s = self.W_Q(Q).view(batch_size_1, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
       k_s = self.W_K(K).view(batch_size_1, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
       v_s = self.W_V(V).view(batch_size_1, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]
      #  print(attn_mask.shape)
       attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

       # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
       context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
       context = context.transpose(1, 2).contiguous().view(batch_size_1, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
#        print(context.device)

       output = self.linear_layer(context)


       return self.layer_norm (output + residual), attn # output: [batch_size x len_q x d_model]

class PoswiseFeedForwardNet(nn.Module):

    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.l1 = nn.Linear(d_model, d_ff)
        self.l2 = nn.Linear(d_ff, d_model)

        self.relu = GELU()
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, inputs):
        residual = inputs
        output = self.l1(inputs)
        output = self.relu(output)
        output = self.l2(output)
        return self.layer_norm(output + residual)

class ScaledDotProductAttention(nn.Module):
   def __init__(self):
       super(ScaledDotProductAttention, self).__init__()

   def forward(self, Q, K, V, attn_mask):
      #  print(Q.shape)
      #  print(K.shape)
      #  print(V.shape)
      #  print(attn_mask.shape)
       scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
      #  print(scores.shape)
      #  print(attn_mask.shape)
       attn_mask = attn_mask.bool()
      #  print(attn_mask.shape)
       scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.

       attn = nn.Softmax(dim=-1)(scores)
       context = torch.matmul(attn, V)
       return  context, attn

class ClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self):
        super(ClassificationHead,self).__init__()
        self.dense = nn.Linear(d_model,d_model)#(config.hidden_size, config.hidden_size)
        classifier_dropout = hidden_dropout_prob       #( config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob     )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(d_model,num_labels)#(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet(d_model=d_model, d_ff=d_ff)
        self.classifier = ClassificationHead()
        self.loss_fct = nn.CrossEntropyLoss()

    def forward(self, enc_inputs, enc_self_attn_mask,labels):
        # print("enter")
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        # print("enc_outputs, attn ")
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        # print("enc_outputs ffn")
        logits = self.classifier(enc_outputs)
        # print("logits")
        loss = self.loss_fct(logits.view(-1, num_labels), labels.view(-1))
        # print("loss")
        return enc_outputs, attn,logits, loss

In [ ]:
encoder_layer = EncoderLayer()

# **Loading the pretrained intent classification Roberta large  model by IBM trained on 180 intents and the tokenizer**

In [ ]:
# Define the BERT model and tokenizer
# keep `ignore_mismatched_sizes=True` so that the classification layer is randomly initialized
# model_name = "cartesinus/bert-base-uncased-amazon-massive-intent"  # Example: You can replace this with the specific RoBERTa variant you want to use
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
model_name="ibm/roberta-large-vira-intents"
# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained(model_name)



In [ ]:
# Initialize the model
num_classes=150
# also outputs hidden state and attention state so that new encoder layer can be added after the last hidden state
model =RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True,output_hidden_states=True,output_attentions=True)
"""since model have its own id2label mapping and vice versa so, converting them from our id2label and label2id mapping as defined earlier"""
model.label2id=label2id
model.id2label=id2label


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ibm/roberta-large-vira-intents and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([181, 1024]) in the checkpoint and torch.Size([150, 1024]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([181]) in the checkpoint and torch.Size([150]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# **Getting the Encodings of the text**

In [ ]:
def add_encodings(example):
  tokenized_inputs = tokenizer(   example['utt'], truncation=True,  is_split_into_words=True   )

  # print(tokenized_inputs)
  encodings = tokenizer(example["utt"], truncation=True, padding='max_length', is_split_into_words=True)
  # print(encodings)
  intent=example["intent"]

  return { **encodings, 'labels': intent }

In [ ]:
train_dataset =  Dataset.from_dict(train_dict)
train_labels = train_dataset['intent']


train_dataset = train_dataset.map(add_encodings)#,

Map:   0%|          | 0/2023 [00:00<?, ? examples/s]

In [ ]:
# train_dataset["attention_mask"].dtype

In [ ]:
valid_dataset =  Dataset.from_dict(test_dict)
valid_labels = valid_dataset['intent']


valid_dataset = valid_dataset.map(add_encodings)#,

Map:   0%|          | 0/225 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
valid_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
valid_dataset

Dataset({
    features: ['utt', 'intent', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 225
})

In [ ]:
train_data = torch.utils.data.DataLoader(train_dataset, batch_size=min_batch_size)
valid_data = torch.utils.data.DataLoader(valid_dataset, batch_size=min_batch_size)

In [ ]:
# Number of batches for each data
print(len(train_data))
print(train_data)
print(len(valid_data))
print(valid_data)

1012
113


In [ ]:
#check cuda or cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model.parameters()
model.to(device) # pushing model to cuda(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [ ]:
encoder_layer.to(device)


EncoderLayer(
  (enc_self_attn): MultiHeadAttention(
    (W_Q): Linear(in_features=1024, out_features=512, bias=True)
    (W_K): Linear(in_features=1024, out_features=512, bias=True)
    (W_V): Linear(in_features=1024, out_features=512, bias=True)
    (linear_layer): Linear(in_features=512, out_features=1024, bias=True)
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (pos_ffn): PoswiseFeedForwardNet(
    (l1): Linear(in_features=1024, out_features=512, bias=True)
    (l2): Linear(in_features=512, out_features=1024, bias=True)
    (relu): GELU()
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (classifier): ClassificationHead(
    (dense): Linear(in_features=1024, out_features=1024, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (out_proj): Linear(in_features=1024, out_features=150, bias=True)
  )
  (loss_fct): CrossEntropyLoss()
)

In [ ]:
encoder_layer

EncoderLayer(
  (enc_self_attn): MultiHeadAttention(
    (W_Q): Linear(in_features=1024, out_features=512, bias=True)
    (W_K): Linear(in_features=1024, out_features=512, bias=True)
    (W_V): Linear(in_features=1024, out_features=512, bias=True)
    (linear_layer): Linear(in_features=512, out_features=1024, bias=True)
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (pos_ffn): PoswiseFeedForwardNet(
    (l1): Linear(in_features=1024, out_features=512, bias=True)
    (l2): Linear(in_features=512, out_features=1024, bias=True)
    (relu): GELU()
    (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (classifier): ClassificationHead(
    (dense): Linear(in_features=1024, out_features=1024, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (out_proj): Linear(in_features=1024, out_features=150, bias=True)
  )
  (loss_fct): CrossEntropyLoss()
)

# **Training the model**

In [ ]:
#run this cell only for training

num_training_steps = n_epochs * len(train_data)#/batch_size

optimizer = optim.AdamW(params=encoder_layer.parameters(), lr=lr)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=5, num_training_steps=num_training_steps)

print("training start")


previous_total_valid_loss = sys.float_info.max
valid_min_loss = previous_total_valid_loss
early_stoping_flag=0
train_loss_epoch=[]
valid_loss_epoch=[]
# Directory where all the weights of model will get saved(Checkpoint)
directory = "/home/naive123/nlp/Sumit/massive/model_1_extra_layer/roberta_ibm"+str(lr)+"_"+str(batch_size)+"_"+str(n_epochs)+"seed"+str(seed)+"/"
# iterate through the data 'n_epochs' times
for epoch in tqdmn(range(n_epochs)):
    # set the model in 'train' mode and send it to the device
    model.to(device)#.train()
    encoder_layer.train().to(device)
    print(epoch)
    print(directory)


    train_loss = []
    current_loss = 0
    total_loss=0
    # iterate through each batch of t    he train data
    for i, batch in enumerate(tqdmn(train_data)):

        # move the batch tensors to the same device as the
        batch = { k: v.to(device) for k, v in batch.items() }
        # print(batch["float_tensor"].dtype)
        batch["attention_mask"]=batch["attention_mask"].unsqueeze(1)
#         print(len(batch["attention_mask"]))
        batch["attention_mask"] = batch["attention_mask"].expand(len(batch["attention_mask"]), d_k, d_k).to(device)
        # print(batch["attention_mask"].shape)#.unsqueeze(1)


#         print(batch["labels"].dtype)
#         print("*")
#         print(len(batch["attention_mask"]))
#         print(len(batch["input_ids"]))
#         print(len(batch["labels"]))

        if len(batch["labels"]) == min_batch_size :
            # send 'input_ids', 'attention_mask' and 'labels' to the model
            outputs = model.forward(**batch)
            # print(outputs)
            enc_outputs, attn, logits, loss = encoder_layer(outputs.hidden_states[24],batch["attention_mask"] , batch["labels"])#**batch outputs.attentions[23]
            if i%200 ==0:
                print(loss)

            # the outputs are of shape (loss, logits)
            #print(outputs[1])


            # loss = outputs[0]

            #print("loss",loss)
            loss.backward()


            current_loss += loss.item()

            if i % batch_size_c == 0 and i > 0: #if i >= 0:  # #           #if i % 8 == 0 and i > 0:
                # update the model using the optimizer
                optimizer.step()
                lr_scheduler.step()
                # once we update the model we set the gradients to zero
                optimizer.zero_grad()
                # store the loss value for visualization
                train_loss.append(current_loss/batch_size)
                total_loss=total_loss + current_loss/batch_size
                current_loss = 0
    train_loss_epoch.append(total_loss)
    # update the model one last time for this epoch
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()

    print(directory + ' files saved')
    print('total_loss',total_loss)
    model.save_pretrained(directory+"best_e_"+str(epoch))
    torch.save(encoder_layer.state_dict(), directory+"best_e_enl"+str(epoch)+'entire_model.pth')
#     torch.save(encoder_layer, directory+"best_e_enl"+str(epoch)+'entire_model.pth')
    #####################################################################################################3
#     validation

    model.eval().to(device)
    encoder_layer.eval().to(device)
    valid_loss = []
    valid_current_loss = 0
    valid_total_loss=0

    # iterate through each batch of t    he train data
    for i, batch in enumerate(tqdmn(valid_data)):

        # move the batch tensors to the same device as the
        batch = { k: v.to(device) for k, v in batch.items() }
        # send 'input_ids', 'attention_mask' and 'labels' to the model
        batch["attention_mask"]=batch["attention_mask"].unsqueeze(1)
#         print(len(batch["attention_mask"]))
        batch["attention_mask"] = batch["attention_mask"].expand(len(batch["attention_mask"]), d_k, d_k).to(device)

        if len(batch["labels"]) == min_batch_size :
            outputs = model(**batch)
            # the outputs are of shape (loss, logits)
            #print(outputs[1])
    #         print("*")
    #         print(batch["attention_mask"].shape)
    #         print(len(batch["attention_mask"]))
    #         print(len(batch["input_ids"]))
    #         print(len(batch["labels"]))

            enc_outputs, attn, logits, loss = encoder_layer(outputs.hidden_states[24],batch["attention_mask"] , batch["labels"])#**batch outputs.attentions[23]
#             print(loss)


            # loss = outputs[0]
            #print("loss",loss)

            if i%25 ==0:
                print(loss)
            valid_current_loss += loss.item()
            #print(valid_current_loss)

            if i % batch_size_c == 0 and i > 0: #depend on batch size and GPU

                valid_loss.append(valid_current_loss/batch_size)
                valid_total_loss=valid_total_loss + valid_current_loss/batch_size
                valid_current_loss = 0
                #print("valid_total_loss ",valid_total_loss)

    print("valid_total_loss ",valid_total_loss)
    valid_loss_epoch.append(valid_total_loss)
    if valid_total_loss < valid_min_loss:
        valid_min_loss = valid_total_loss
#         model.save_pretrained(directory+"best_e_"+str(epoch))


    if previous_total_valid_loss < valid_total_loss:
        early_stoping_flag += 1
    else:
        early_stoping_flag=0
    previous_total_valid_loss = valid_total_loss


    if early_stoping_flag == early_stoping_patience:
        print("early_stoping occure")
        break



print('training complete')





training start


  0%|          | 0/30 [00:00<?, ?it/s]

0
/home/naive123/nlp/Sumit/massive/model_1_extra_layer/roberta_ibm0.0006_32_30seed50/


  0%|          | 0/1012 [00:00<?, ?it/s]

tensor(5.0678, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.8293, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.3078, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.8448, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1009, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7810, device='cuda:0', grad_fn=<NllLossBackward0>)
/home/naive123/nlp/Sumit/massive/model_1_extra_layer/roberta_ibm0.0006_32_30seed50/ files saved
total_loss 82.2145941236522


...

tensor(0.0008, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0064, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0252, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0030, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0033, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0046, device='cuda:0', grad_fn=<NllLossBackward0>)
/home/naive123/nlp/Sumit/massive/model_1_extra_layer/roberta_ibm0.0006_32_30seed50/ files saved
total_loss 2.26468389922411


  0%|          | 0/113 [00:00<?, ?it/s]

tensor(0.0060, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.1071, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0238, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2500, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0039, device='cuda:0', grad_fn=<NllLossBackward0>)
valid_total_loss  2.2309623442397424
12
/home/naive123/nlp/Sumit/massive/model_1_extra_layer/roberta_ibm0.0006_32_30seed50/


  0%|          | 0/1012 [00:00<?, ?it/s]

tensor(0.0012, device='cuda:0', grad_fn=<NllLossBackward0>)


# **For prediction at test data **


In [ ]:
# Prediction
def  prediction(best_model_path,best_encoder_path, eval_data):

    predicted=[]
    tokanised_exam=[]
    true_labels=[]

    predicted_for_classification_report=[]

    ##################################################
    #load best model


    print(best_model_path)

    model = RobertaForSequenceClassification.from_pretrained(best_model_path)
    model = model.eval().to(device)

    encoder_layer =EncoderLayer()

    encoder_layer=encoder_layer.to(device)
    encoder_layer.load_state_dict(torch.load(best_encoder_path))
    encoder_layer.eval()
    print("model device")
    # batch the train data so that each batch contains 4 examples (using 'batch_size')
    #test_data = torch.utils.data.DataLoader(test_dataset, batch_size=16)

    eval_data = eval_data



    # iterate through each batch of the eval data
    for i, batch in enumerate(tqdmn(eval_data)):



       # move the batch tensors to the same device as the
        batch = { k: v.to(device) for k, v in batch.items() }
        # send 'input_ids', 'attention_mask' and 'labels' to the model
        batch["attention_mask"]=batch["attention_mask"].unsqueeze(1)
#         print(len(batch["attention_mask"]))
        batch["attention_mask"] = batch["attention_mask"].expand(len(batch["attention_mask"]), d_k, d_k).to(device)

        if len(batch["labels"]) == min_batch_size :
            outputs = model(**batch)
            print(outputs.hidden_states[24].shape)
            # the outputs are of shape (loss, logits)
            #print(outputs[1])
    #         print("*")
    #         print(batch["attention_mask"].shape)
    #         print(len(batch["attention_mask"]))
    #         print(len(batch["input_ids"]))
    #         print(len(batch["labels"]))

            enc_outputs, attn, logits, loss = encoder_layer(outputs.hidden_states[24],batch["attention_mask"] , batch["labels"])#**batch outputs.attentions[23]
#             print(loss)

        print(logits.shape)
        # iterate through the examples

        # Apply softmax to get class probabilities
#         probabilities = torch.softmax(logits, dim=1)

        # Get the predicted class (index with the highest probability)
        pred_values = torch.argmax(logits, dim=1)
        pred_values = pred_values.tolist()
        predicted.append(pred_values)

        true_values = batch['labels']
        print(true_values.shape)
        true_labels.extend(true_values.tolist())

        predicted_for_classification_report.extend(pred_values)


          # go through all true and predicted values and store them in the confusion matrix
          # for true, pred in zip(true_values, pred_values):
          #     confusion[true.item()][pred.item()] += 1
    print(predicted_for_classification_report)
    return predicted, predicted_for_classification_report


In [ ]:
#**Put path of best checkponts saved during training**
best_encoder_path="/home/naive123/nlp/Sumit/massive/model_1_extra_layer/roberta_ibm0.0001_32_22seed55/best_e_enl2entire_model.pth"

best_model_path="/home/naive123/nlp/Sumit/massive/model_1_extra_layer/roberta_ibm0.0001_32_22seed55/best_e_2/"

In [ ]:
predicted, predicted_for_classification_report = prediction(best_model_path, best_encoder_path,valid_data)

In [ ]:
len(predicted_for_classification_report)
# target_names = labels_list
from sklearn.metrics import classification_report
print(classification_report(test_labels, predicted_labels[:-1]))#,labels=[i for i in range(150)]))


In [ ]:
len(test_labels)
test_labels

In [ ]:
predicted_labels=[id2label[i] for i in predicted_for_classification_report]
predicted_labels

# **making pytorch datasets**

# **Blind dataset**

In [ ]:
test_list[:20]

# **loading the tokenizer for predicting labels on blind dataset(whose labels not available)**

In [ ]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(model_name)
test_text = [item["utt"] for item in test_list]

# Tokenize the test data
test_set_encodings = tokenizer(test_text, padding=True, truncation=True, return_tensors="pt")


In [ ]:
test_set_encodings

In [ ]:
"""Dataloader and Also Dataset to convert the testing(blind) dataset featues(encodings) obtained from the tokenizer into the format suitable for the training"""
from torch.utils.data import DataLoader, TensorDataset

# Convert the encodings into tensors
input_ids = test_set_encodings['input_ids']
attention_mask = test_set_encodings['attention_mask']
# token_type_ids = test_set_encodings['token_type_ids']

# Create a TensorDataset
test_dataset = TensorDataset(input_ids, attention_mask,)

# Define batch size
batch_size = 32

# Create a DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)


# **prediction at blind test data set**

In [ ]:
predicted, predicted_for_classification_report = prediction(best_model_path, best_encoder_path,test_dataloader)

# **Preparing For submission format**

In [ ]:
predicted_labels_test=[id2label[int(id)] for id in list(predictions)]
predicted_labels_test

In [ ]:
import pandas as pd
my_id=[]
my_intent1=[]

my_dict1={}
for i, entry in enumerate(test_list):
    my_id.append({'indoml_id':i+1,'intent':predicted_labels_test[i]})
    my_intent1.append(predicted_labels_test[i])
    my_dict1['id']=my_id
    my_dict1['intent1']=my_intent1
#     temp={}
#     temp['indoml_id']=entry['locale']+'/'+entry['id']
#     temp['intent']=entry['pred_intent']
#     my_dict['k']=temp
#     k+=1
my_dict1_pd=pd.DataFrame.from_dict(my_dict1)
my_dict1_pd.head()


In [ ]:
with open('output.predict', 'w') as out_file:
    for entry in my_dict1_pd['id']:
        out_file.write(str(entry))
        out_file.write('\n')
